# Base64 encoding MyPHD keys 

This script leverages PyCap to do the following:

1. Downloads MyPHD keys from a given project
2. Encodes them as base64
3. Uploads resulting string as a field for each record

### Setup



In [ ]:
# Create and activate virtual environment
! python3 -m venv venv
! source ./venv/bin/activate venv

# Install kernel 
! pip install ipykernel
! python3 -m ipykernel install --user 

# Create environmental file, follow the example to fill in the necessary variables
! touch .env

# If you are using VsCode: Ensure in python3 venv interpreter is selected with Ctrl+Shift+P

### Install dependencies
- Pycap is the package used to integrate with the REDCap API
- Dotenv is a package used to handle environmental variables

In [ ]:

! pip install pycap
! pip install python-dotenv

## Script Execution

### Import dependencies

In [ ]:
import os
import base64
from datetime import datetime
from dotenv import load_dotenv
from redcap import Project, RedcapError
load_dotenv()

### Default variable fields
These fields can be set according to your REDCap project

In [90]:
recordPrimaryKey = 'record_id' # General ID field of record
recordFileField = 'key' # Field to pull from
recordUploadField = 'key_base64' # Field to upload resulting data to

In [ ]:
error = []

if not os.getenv('REDCAP_API_KEY') or not os.getenv('REDCAP_API_URL'):
    error.append('Environmental variables have not been set. Please update your .env file')
else: 
    try:
        
        # Initialize connection to project
        project = Project(str(os.getenv('REDCAP_API_URL')), str(os.getenv('REDCAP_API_KEY')))
        
        # Grab all record IDs
        records = project.export_records(
            format_type='json', 
            fields=[recordPrimaryKey], 
            raw_or_label='raw'
        )
        newRecords = []
        
        for record in records: 
            if(int(record[recordPrimaryKey])):
                # Grab key file from record
                key = project.export_file(
                    record=record[recordPrimaryKey], 
                    field=recordFileField
                )
                
                if key: 
                    # encode as base64, convert to string representation and store
                    encoded = base64.b64encode(key[0])
                    newRecord = {
                        recordPrimaryKey: int(record[recordPrimaryKey]),
                        recordUploadField: encoded.decode('ascii')
                    }
                    newRecords.append(newRecord)
                else:
                    error.append(f'Record {record[recordPrimaryKey]} has no file uploaded with the field name {recordFileField}')
        
        # import all new records
        project.import_records(newRecords)
        
    except RedcapError as e:
        error.append(f'REDCap Error : {e}')
    except ValueError as e: 
        error.append(f'Value Error : {e}')
    
# Log errors if any
if len(error):
    time = datetime.now().strftime('%B %d, %Y %H:%M:%S')
    with open('error.log', 'a') as f:
        f.write(f'\n\n####### Error log for script execution on {time}\n')
        f.write('\n'.join(error))
    